### Pytorch to TFLite Conversion Guide
This is a simple guide to help conversion from Pytorch model to TFLite. The network weights is transferred from Pytorch model to Keras model, then converted into TFLite. <br>
**Note : this guide does not contains automatic conversion, instead the model must be re-write into keras model.** <br>
There are existing method to convert Pytorch model to Tensorflow via ONNX for example. I tried to use for my project but it does not work, so I figured a work around by rewriting the model manually.

In [ ]:
import os
import sys
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras.layers as kl
from tensorflow.keras.models import *

We define simple convolutional model with Pytorch and Keras model. Ideally for transfering the weight, the operation name between two model should be same. <br>
However any name model should work with some modification.
<pre>
(Pytorch) - (Keras)
  conv1   -  conv1
  conv2   -  conv2
</pre>

In [ ]:
# Simple Pytorch model
class PyModel(nn.Module):
    def __init__(self):
        super(PyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, (3,3), 1, 1)
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, (3,3), 1,1),
            nn.LeakyReLU(0.2),
        )
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        return out
    
# Simple Keras model
def KModel(inp_shape=[200, 200, 3]):
    x = kl.Input(inp_shape)
    net = kl.ZeroPadding2D((1,1))(x)
    net = kl.Conv2D(64, 3, strides=(1,1), padding='valid', name='conv1')(net)
    net = kl.ZeroPadding2D((1,1))(net)
    net = kl.Conv2D(64, 3, strides=(1,1), padding='valid', name='conv2')(net)
    net = kl.LeakyReLU(0.2)(net)
    model = Model(inputs=x, outputs=net)
    return model

To transfer the weights, we define a function to load all weight and bias parameters into dictionary.
The dictionary is used to transfer the weight to keras model within the same name. <br>
Pytorch convolutional layer consists of weight and bias class attributes. Keras model defines the weight and bias using `list []`. <br>
Keras `padding='same'` will result in different output with pytorch, therefore we must add padding to the convolutional layer.

In [ ]:
def transfer_weights(pymodel, kmodel):
    # Create a weights dictionary
    weights = {}
    
    # Loop over the pytorch model and store in dict
    for name, module in pymodel._modules.items():
        if isinstance(module, nn.Conv2d):
            # Take weight and bias of convolutional model
            # For the purpose of testing, we are setting the weight to be constant 1 and 0
            w = module.weight.data.numpy()
            b = module.bias.data.numpy()
            # Store weight in bias in the dict
            weights[name] = [w,b]
        # Handle Sequential model
        if isinstance(module, nn.Sequential):
            for m in module:
                if isinstance(m, nn.Conv2d):
                    w = m.weight.data.numpy()
                    b = m.bias.data.numpy()
                    weights[name] = [w,b]

    # Loop over the keras model
    for lyr in kmodel.layers:
        # Check if it contains trainable weights
        if len(lyr.get_weights()) > 0:
            if lyr.name in weights.keys():
                # Set weights from dict and transpose the 'w' from NCHW - NHWC
                [w,b] = weights[lyr.name]
                lyr.set_weights([w.transpose(2,3,1,0), b])
            else:
                print("Missing", lyr.name, "key in dictionary")

Make sure to use `tensorflow.keras` instead of `keras`, or the model cannot be converted to TFLite format. <br>
The current function is written to handle direct and sequential module only. To handle different variations like nested module, the function must be modified. The example is shown, below

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, inch, outch, ksz, st, pad):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(inch, outch, ksz, st, pad),
            nn.ReLU(),
            nn.InstanceNorm2d(outch),
        )
    def forward(self, x):
        out = self.block(x)
        return out

class PyModelBlock(nn.Module):
    def __init__(self):
        super(PyModelBlock, self).__init__()
        self.conv_block1 = ConvBlock(3, 64, (3,3), 1, 1)
        self.conv_blcok2 = ConvBlock(64, 64, (3,3), 1, 1)
    
    def forward(self, x):
        out = self.conv_block1(x)
        out = self.conv_block2(out)
        return out

Then to access the weights and bias, we need to got into the main module class name and access the nested module class name.
```python
for name, module in pymodel._modules.items():
    main_module = module.block # Sequential module defines in `ConvBlock`
    # or main_module[i] to access sequential module
    for mod in main_module:
        if isinstance(mod, nn.Conv2d):
            w = mod.weight.data.numpy()
            b = mod.bias.dataa.numpy()
```
The idea is to loop over the pytorch model and store every weights. You'll get the idea. <br>
We use `_modules.items()`, but we can use any method to loop over the model like `named_parameters()` or `child()`.

### Converting the model to TFLite
To convert the model to TFLite, we will use `TFLiteConverter` from tensorflow. It can handle different model such as keras model files.

In [ ]:
# Define the Pytorch and Keras model
pmodel = PyModel()
kmodel = KModel()
# If we have pretrained Pytorch model, load them
# pmodel.load_state_dict(torch.load(SOME_PYTORCH_MODEL))

Next we transfer weights using the transfer weights function. We save the model into keras model format.

In [ ]:
transfer_weights(pmodel,kmodel)
tf.keras.models.save_model(kmodel, '../models/keras_model.h5')

### Verifying The Converted Model
The conversion is finished until here. We need to verify if the model is converted successfully and yield same result with the Pytorch model.

In [ ]:
kmodel_ = tf.keras.models.load_model('../models/keras_model.h5')

Prepare a dummy input for verification and forward the model.

In [ ]:
inp = torch.rand(1, 3, 200, 200) # NCHW
inp_k = inp.numpy().transpose(0,2,3,1) # NHWC

out_pytorch = pmodel(inp)
out_pytorch = out_pytorch.detach().numpy().transpose(0,2,3,1)
out_keras = kmodel_.predict(inp_k, batch_size=1)

Verify the output from both models.

In [9]:
print('Pytorch:', out_pytorch.shape, 'Keras:', out_keras.shape)
print('Mean difference : %.20f' % np.mean(np.abs(out_keras - out_pytorch)))

Pytorch: (1, 200, 200, 64) Keras: (1, 200, 200, 64)
Mean difference : 0.00000005061199814804


We successfully converted the model to keras by transferring the weights.

### TFLite Conversion
We convert the keras model to TFLite format and save it to file.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model_file('../models/keras_model.h5')
tfmodel = converter.convert()
with open('../models/tflite_model.tflite', 'wb') as f: 
    f.write(tfmodel)

# You can compile the keras model before the conversion if you desired, just in case.

TFLite can be run in python, C++, ios and android. For the android tutorial using TFLite please check [here](./Android_TFLite_tutorial.md). <br>
The last remaining step is to verify our tflite model, we will run the tflite inference using python.

In [ ]:
net = tf.lite.Interpreter(model_path='../models/tflite_model.tflite')
net.allocate_tensors()

inp_details = net.get_input_details() # Get the tensor operation name (dictionary)
out_details = net.get_output_details() 

net.set_tensor(inp_details[0]['index'], value=inp_k)
net.invoke()
out = net.get_tensor(out_details[0]['index'])

Verify the output of tflite model.

In [12]:
print('Pytorch:', out_pytorch.shape, 'TFLite', out.shape)
print('Mean difference : %.20f' % np.mean(np.abs(out - out_pytorch)))

Pytorch: (1, 200, 200, 64) TFLite (1, 200, 200, 64)
Mean difference : 0.00000005379431655683


We successfully converted the model to TFLite with.